In [62]:
import pandas as pd
import numpy as np
from datetime import *
import networkx as nx
from collections import *

In [2]:
def dateparse(time_as_a_unix_timestamp):
    return pd.to_datetime(time_as_a_unix_timestamp, unit="s").strftime("%Y-%m-%d %H:%M")

In [3]:
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 1. Data

In [23]:
#Answers to questions
a2q = pd.read_csv("sx-stackoverflow-a2q.txt", sep=" " ,header=None, names=["user_a", "user_b", "time"], parse_dates=["time"], date_parser=dateparse)

#Comments to answers
c2a = pd.read_csv("sx-stackoverflow-c2a.txt", sep=" " ,header=None, names=["user_a", "user_b", "time"], parse_dates=["time"], date_parser=dateparse)

#Comments to questions
c2q = pd.read_csv("sx-stackoverflow-c2q.txt", sep=" " ,header=None, names=["user_a", "user_b", "time"], parse_dates=["time"], date_parser=dateparse)

KeyboardInterrupt: 

In the merged graph we have weighted link and we have decided to build these weights in the following way:
-  we have assigned a score of 1.0 to "Answers to questions"
-  we have assigned a score of 0.7 to "Comments to questions"
-  we have assigned a score of 0.4 to "Comments to answers"

The reasons are that we have thought that the answers to questions are the more relevant in the merged graph because they are possible solutions of user's question. Then a bit less important are comments to questions, but they could be relevant to clarify the question or specify something. Comments to answer are the least relevant, but we have not wanted to penalize them too much because in some situations they could be useful.

In [44]:
#Answers to questions
a2q["weights"] = 1

#Comments to answers
c2a["weights"] = 0.4

#Comments to questions
c2q["weights"] = 0.7

Nice and simultaneous visualization of the three file!

In [26]:
display("a2q", "c2a", "c2q")

,user_a,user_b,time
0,9,8,2008-08-01 05:17:00
1,1,1,2008-08-01 06:56:00
2,13,1,2008-08-01 15:57:00
3,17,1,2008-08-01 19:07:00
4,48,2,2008-08-01 19:16:00
...,...,...,...
17823520,2773607,1048138,2016-03-06 12:16:00
17823521,6018278,1982354,2016-03-06 12:16:00
17823522,3187183,1404306,2016-03-06 12:17:00
17823523,6022341,1667278,2016-03-06 12:17:00


In [27]:
totDataframe = pd.concat([a2q_1, c2a_1, c2q_1])
totDataframe

,user_a,user_b,time,weights
0,9,8,2008-08-01 05:17:00,1.0
1,1,1,2008-08-01 06:56:00,1.0
2,13,1,2008-08-01 15:57:00,1.0
3,17,1,2008-08-01 19:07:00,1.0
4,48,2,2008-08-01 19:16:00,1.0
...,...,...,...,...
20268146,1177890,4917254,2016-03-06 14:06:00,0.7
20268147,3345375,4022289,2016-03-06 14:08:00,0.7
20268148,3507137,1801524,2016-03-06 14:09:00,0.7
20268149,5617035,4368648,2016-03-06 14:10:00,0.7


In [42]:
class EDGE:
    
    def __init__(self, from_n, to_n, time, w):
        self.from_n = from_n
        self.to_n = to_n
        self.w = w
        self.time = time
        
    def __repr__(self):
        return f"{self.from_n} -> {self.to_n} ::::: weight = {self.w} ::::: time: {self.time}"

In [7]:
class GRAPH:
    
    def __init__(self, nodes = [], edges = []):
        self.nodes = defaultdict(list, {k:[] for k in nodes})
        
        if(len(edges)>0):
            for edge in edges:
                self.nodes[edge.from_n] = edge
                self.nodes[edge.to_n] = edge
        
        self.num_nodes = len(nodes)
        self.num_edges = len(edges)
        
    def add_edge(self, from_n, to_n, w, time):
        e = EDGE(from_n, to_n, w, time)
        
        self.nodes[from_n].append(e)
        self.nodes[to_n].append(e)

    def __repr__(self):
        return f"{self.nodes}"

## Functionality 1 - Get the overall features of the graph

In [34]:
#array of nodes of graph a2q
nodi_grafo = pd.concat([a2q_1["user_a"], a2q_1["user_b"]], axis = 0).drop_duplicates().array


<PandasArray>
[      9,       1,      13,      17,      48,      19,      23,      35,
      39,      43,
 ...
 6023235, 6024201, 4589438, 6023919, 6024094, 6024193, 6024119, 6023749,
 4411955, 6024256]
Length: 2464606, dtype: int64

In [48]:
for i in range(3): 
     print(a2q.iloc[i,:])

user_a                       9
user_b                       8
time       2008-08-01 05:17:00
weights                      1
Name: 0, dtype: object
user_a                       1
user_b                       1
time       2008-08-01 06:56:00
weights                      1
Name: 1, dtype: object
user_a                      13
user_b                       1
time       2008-08-01 15:57:00
weights                      1
Name: 2, dtype: object


In [46]:
EDGE(a2q.iloc[0,0],a2q.iloc[0,1], a2q.iloc[0,2], a2q.iloc[0,3])

9 -> 8 ::::: weight = 1 ::::: time: 2008-08-01 05:17:00

In [57]:
edge_grafo = []
for i in range(len(a2q)): 
    edge_grafo.append(EDGE(a2q.iloc[i,0],a2q.iloc[i,1], a2q.iloc[i,2], a2q.iloc[i,3]))

In [58]:
#creation of the graph a2q
a2q_graph = GRAPH(nodi_grafo, edge_grafo)

In [72]:
#Number of users
num_node = a2q_graph.num_nodes
num_node

2464606

In [68]:
#Number of answers
num_edge = a2q_graph.num_edges
num_edge

17823525

In [69]:
# check if the source node is also in the dest nodes, in order to check if the graph is directed or not
#in an undirected graph the number of starting nodes is the same of the ending nodes

if len(set(list(a2q["user_a"])).intersection(set(list(a2q["user_b"])))) > 0: 
    direct = "directed"
else:
    direct = "undirected"
print(direct)

directed


In [92]:
#Average number of links per user
len(a2q_graph.nodes.keys())

2464606

In [81]:
end = 0
while (end == 0):
    
    user = int(input("Welcome user, what functionality do you want: "))
    if (user == 1):
        
        print("Functionality 1 - Get the overall features of the graph")
        print("The a2q is: " + direct)
        print("Number of users: " +str(num_node))
        print("Number of answers: " +str(num_edge))
        print("result of func_1")
        end += 1
        
    if (user == 2):
        
        print("result of func_2")
        print("Functionality 2 - Find the best users!")
        end += 1
        
    if (user == 3):
        print("Functionality 3 - Shortest Ordered Route")
        print("result of func_3")
        end += 1


KeyboardInterrupt: Interrupted by user

In [82]:
2+4

6